# entrainenement NLP
disclaimer : ceci un notepad pour s'entrainer au NLP

In [1]:
import pandas as pd
from functionDataFrame import *

df = readDataframe()

df["text"] = df["description"] + " " + df["claim"]

In [1]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [ ]:
from transformers import BertTokenizer, BertModel
import torch

# Charger le modèle et le tokenizer BERT pré-entraînés
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased').to(device)

def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy()

# Appliquer BERT sur la colonne de texte pour obtenir les embeddings
df['embeddings'] = df['text'].apply(get_bert_embeddings)



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Assurez-vous que CPC_code est la variable cible, encodée en entiers
X = list(df['embeddings'])
y = df['CPC_code']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraîner un classificateur de régression logistique
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

# Prédire sur l'ensemble de test
y_pred = clf.predict(X_test)

# Évaluer le classificateur
print(classification_report(y_test, y_pred))


In [ ]:
# Assumant que vous avez un nouveau DataFrame `new_df` avec la même structure
new_df = pd.read_csv('path_to_your_new_data.csv')
new_df['text'] = new_df['description'] + " " + new_df['claims']
new_df['embeddings'] = new_df['text'].apply(get_bert_embeddings)

# Prédire les codes CPC pour les nouvelles données
new_X = list(new_df['embeddings'])
new_df['predicted_CPC_code'] = clf.predict(new_X)


In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# Préparer les étiquettes
label_list = df['CPC_code'].unique()
num_labels = len(label_list)

# Charger le modèle BERT pour la classification de séquences
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)

# Préparer le tokenizer
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=True, max_length=512)

# Charger le Dataset
from datasets import Dataset
dataset = Dataset.from_pandas(df)

# Tokeniser les données
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Configurer les arguments d'entraînement
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Créer le Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets
)

# Entraîner le modèle
trainer.train()
